# Quokka

In [284]:
import pandas as pd
import tensorflow as tf
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split


In [285]:
df = pd.read_csv('datasets/forestfires.csv')

In [286]:
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [287]:
df.count()

X        517
Y        517
month    517
day      517
FFMC     517
DMC      517
DC       517
ISI      517
temp     517
RH       517
wind     517
rain     517
area     517
dtype: int64

In [288]:
df.drop(['X', 'Y', 'day'], axis=1, inplace=True)

In [289]:
# Dictionary mapping month names to month numbers
month_to_number = {
    'jan': 1,
    'feb': 2,
    'mar': 3,
    'apr': 4,
    'may': 5,
    'jun': 6,
    'jul': 7,
    'aug': 8,
    'sep': 9,
    'oct': 10,
    'nov': 11,
    'dec': 12
}

# Convert month names to month numbers using map() function
df['month'] = df['month'].map(month_to_number)


In [290]:
X = df[['month', 'temp', 'RH', 'wind', 'rain']]
Y = (df['area'] > 0).astype(float)

In [291]:
X.shape

(517, 5)

In [292]:
X.dtypes

month      int64
temp     float64
RH         int64
wind     float64
rain     float64
dtype: object

In [293]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)


In [294]:
X_test


,month,temp,RH,wind,rain
296,6,14.3,46,1.8,0.0
439,9,19.1,46,2.2,0.0
170,9,21.9,35,1.8,0.0
339,9,20.4,41,1.8,0.0
141,8,23.3,31,4.5,0.0
363,9,15.9,53,2.2,0.0
210,10,18.4,25,3.1,0.0
44,9,12.9,74,4.9,0.0
38,10,17.8,27,4.0,0.0
418,8,24.9,42,5.4,0.0


In [295]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(5,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)


In [296]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [297]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_66 (Dense)            (None, 64)                384       
                                                                 
 dense_67 (Dense)            (None, 32)                2080      
                                                                 
 dense_68 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,497
Trainable params: 2,497
Non-trainable params: 0
_________________________________________________________________


In [298]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
15/15 [==============================] - 9s 314ms/step - loss: 0.9177 - accuracy: 0.4710 - val_loss: 0.8035 - val_accuracy: 0.3462
Epoch 2/100
15/15 [==============================] - 3s 219ms/step - loss: 0.7260 - accuracy: 0.5161 - val_loss: 0.7533 - val_accuracy: 0.5192
Epoch 3/100
15/15 [==============================] - 1s 88ms/step - loss: 0.7534 - accuracy: 0.4989 - val_loss: 0.9555 - val_accuracy: 0.2885
Epoch 4/100
15/15 [==============================] - 1s 85ms/step - loss: 0.7678 - accuracy: 0.5032 - val_loss: 0.8825 - val_accuracy: 0.2885
Epoch 5/100
15/15 [==============================] - 1s 89ms/step - loss: 0.7045 - accuracy: 0.5355 - val_loss: 0.7484 - val_accuracy: 0.3654
Epoch 6/100
15/15 [==============================] - 1s 88ms/step - loss: 0.6993 - accuracy: 0.5355 - val_loss: 0.6654 - val_accuracy: 0.6538
Epoch 7/100
15/15 [==============================] - 1s 92ms/step - loss: 0.7199 - accuracy: 0.5011 - val_loss: 0.6472 - val_accuracy: 0.6923
Epoc

In [311]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss:.2f}')
print(f'Accuracy: {accuracy:.2f}')

# Make predictions on new data
# [[month, temp, RH, wind, rain]]
# some test samples:
# [[6, 40, 20, 4.3,0.0]]
# [[12, 10, 70, 0,0]]
# [[1, 20, 50, 2.2,0.0]]

new_data = [[6, 40, 20, 4.3,0.0]]
predictions = model.predict(new_data)

# prediction in range 0 - 1
print(f'Chance of forest fire: {1 - predictions[0][0]:3f}')

2/2 [==============================] - 1s 5ms/step - loss: 0.6511 - accuracy: 0.6923
Loss: 0.65
Accuracy: 0.69
1/1 [==============================] - 1s 1s/step
Chance of forest fire: 0.532214


In [320]:
## multiple prediction
import random as rand
new_data = [(rand.randint(1,12), rand.randint(0,40), rand.randint(0,100), rand.randint(0,10), rand.randint(0,10)) for i in range(100)]

predictions = model.predict(new_data)
for i in range(len(new_data)):
    print(f"month: {new_data[i][0]} temp: {new_data[i][1]} RH: {new_data[i][2]} wind: {new_data[i][3]} rain: {new_data[i][4]}\nchance of forest fire: {1-predictions[i][0]:3f}\n")

4/4 [==============================] - 4s 3ms/step
month: 12 temp: 33 RH: 61 wind: 0 rain: 6
chance of forest fire: 0.016458

month: 12 temp: 29 RH: 20 wind: 10 rain: 9
chance of forest fire: 0.056810

month: 1 temp: 28 RH: 42 wind: 6 rain: 9
chance of forest fire: 0.019480

month: 12 temp: 20 RH: 71 wind: 6 rain: 2
chance of forest fire: 0.157823

month: 9 temp: 30 RH: 50 wind: 8 rain: 2
chance of forest fire: 0.269709

month: 3 temp: 33 RH: 84 wind: 0 rain: 8
chance of forest fire: 0.022058

month: 10 temp: 34 RH: 13 wind: 5 rain: 3
chance of forest fire: 0.348072

month: 10 temp: 1 RH: 38 wind: 3 rain: 3
chance of forest fire: 0.002961

month: 4 temp: 13 RH: 100 wind: 0 rain: 9
chance of forest fire: 0.136122

month: 3 temp: 22 RH: 34 wind: 2 rain: 3
chance of forest fire: 0.123497

month: 6 temp: 6 RH: 13 wind: 9 rain: 1
chance of forest fire: 0.029742

month: 9 temp: 10 RH: 52 wind: 3 rain: 7
chance of forest fire: 0.034228

month: 11 temp: 19 RH: 0 wind: 3 rain: 10
chance of fore

In [308]:
model.save('models/quokkaForesterV1.0.h5')